In [2]:
BOARD_EMPTY = 0
BOARD_PLAYER_X = 1
BOARD_PLAYER_O = -1

In [8]:
# Commented out the invalid line as it is not valid Python and is causing the error.
# [ 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ]
# ^   ^   ^   ^   ^   ^   ^   ^   ^

# Valid Python list
grid = [0, 0, 0, 0, 0, 0, 0, 0, 0]

# Positions mapping
# tl  tm  tr  ml  mm  mr  bl  bm  br
# t ~ top
# m ~ middle
# b ~ bottom
# l ~ left
# r ~ right

# Defining positions
positions = {
    "tl": 0, "tm": 1, "tr": 2,
    "ml": 3, "mm": 4, "mr": 5,
    "bl": 6, "bm": 7, "br": 8
}

# Example function to update a position
def update_position(grid, position, value):
    if position in positions:
        grid[positions[position]] = value
    else:
        print("Invalid position")

# Example usage
update_position(grid, "mm", 1)
print(grid)

# Function to display the grid in a 3x3 format
def display_grid(grid):
    for i in range(0, 9, 3):
        print(grid[i:i+3])

# Display the grid
display_grid(grid)


[0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0]
[0, 1, 0]
[0, 0, 0]


In [9]:
def print_board(s):
  def convert(num):
    if num == BOARD_PLAYER_X:
      return 'X'
    if num == BOARD_PLAYER_O:
      return 'O'
    return '_'

  i = 0
  for _ in range(3):
    for _ in range(3):
      print(convert(s[i]), end=' ')
      i += 1
    print()

In [10]:
from collections import Counter

def player(s):
  counter = Counter(s)
  x_places = counter[1]
  o_places = counter[-1]

  if x_places + o_places == 9:
    return None
  elif x_places > o_places:
    return BOARD_PLAYER_O
  else:
    return BOARD_PLAYER_X

In [11]:
def actions(s):
  play = player(s)
  actions_list = [(play, i) for i in range(len(s)) if s[i] == BOARD_EMPTY]
  return actions_list

In [12]:
def result(s, a):
  (play, index) = a
  s_copy = s.copy()
  s_copy[index] = play
  return s_copy

In [13]:
def terminal(s):
  for i in range(3):
    # Checking if a row is filled and equal.
    if s[3 * i] == s[3 * i + 1] == s[3 * i + 2] != BOARD_EMPTY:
      return s[3 * i]
    # Checking if a column is filled and equal.
    if s[i] == s[i + 3] == s[i + 6] != BOARD_EMPTY:
      return s[i]

  # Checking if a diagonal is filled and equal.
  if s[0] == s[4] == s[8] != BOARD_EMPTY:
    return s[0]
  if s[2] == s[4] == s[6] != BOARD_EMPTY:
    return s[2]

  # Checking if the game has no more moves available
  if player(s) is None:
    return 0

  # Return None if none of the previous conditions satisfy.
  return None

In [14]:
def utility(s):
  term = terminal(s)
  # Return who wins the game if the game has terminated
  if term is not None:
    return term

  # Get the list of actions available
  action_list = actions(s)
  utils = []
  for action in action_list:
    # Create a new state applying the action to current state
    new_s = result(s, action)
    # Add the score of the new state to a list
    utils.append(utility(new_s))

  score = utils[0]
  play = player(s)
  # Calculate the max score if X is playing
  if play == BOARD_PLAYER_X:
    for i in range(len(utils)):
      if utils[i] > score:
        score = utils[i]
  # Calculate the min score if O is playing
  else:
    for i in range(len(utils)):
      if utils[i] < score:
        score = utils[i]
  return score

In [15]:
def utility(s, cost):
  term = terminal(s)
  if term is not None:
    # Return the cost of reaching the terminal state
    return (term, cost)

  action_list = actions(s)
  utils = []
  for action in action_list:
    new_s = result(s, action)
    # Every recursion will be an increment in the cost (depth)
    utils.append(utility(new_s, cost + 1))

  # Remember the associated cost with the score of the state.
  score = utils[0][0]
  idx_cost = utils[0][1]
  play = player(s)
  if play == BOARD_PLAYER_X:
    for i in range(len(utils)):
     if utils[i][0] > score:
       score = utils[i][0]
       idx_cost = utils[i][1]
  else:
    for i in range(len(utils)):
      if utils[i][0] < score:
        score = utils[i][0]
        idx_cost = utils[i][1]

  # Return the score with the associated cost.
  return (score, idx_cost)

In [16]:
def minimax(s):
  action_list = actions(s)
  utils = []
  for action in action_list:
    new_s = result(s, action)
    utils.append((action, utility(new_s, 1)))
  # Each item in utils contains the action associated
  # the score and "cost" of that action.

  # if utils has no objects, then return a default action and utility
  if len(utils) == 0:
    return ((0, 0), (0, 0))

  # Sort the list in ascending order of cost.
  sorted_list = sorted(utils, key=lambda l : l[0][1])
  # Since the computer shall be Player O,
  # It is safe to return the object with minimum score.
  action = min(sorted_list, key = lambda l : l[1])
  return action

In [18]:
if __name__ == '__main__':
  # Initializing the state
  s = [BOARD_EMPTY for _ in range(9)]
  print('|------- WELCOME TO TIC TAC TOE -----------|')
  print('You are X while the Computer is O')

  # Run the program while the game is not terminated
  while terminal(s) is None:
    play = player(s)
    if play == BOARD_PLAYER_X:
      # Take input from user
      print('\n\nIt is your turn', end='\n\n')
      x = int(input('Enter the x-coordinate [0-2]: '))
      y = int(input('Enter the y-coordinate [0-2]: '))
      index = 3 * x + y

      if not s[index] == BOARD_EMPTY:
        print('That coordinate is already taken. Please try again.')
        continue

      # Apply the action and print the board
      s = result(s, (BOARD_PLAYER_X, index))
      print_board(s)
    else:
      print('\n\nThe is computer is playing its turn')
      # Get the action by running the minimax algorithm
      action = minimax(s)
      # Apply the returned action to the state and print the board
      s = result(s, action[0])
      print_board(s)

  # We know that terminal(s) is not None
  # determine the winner
  winner = terminal(s)
  if winner == BOARD_PLAYER_X:
    print("You have won!")
  elif winner == BOARD_PLAYER_O:
    print("You have lost!")
  else:
    print("It's a tie.")

|------- WELCOME TO TIC TAC TOE -----------|
You are X while the Computer is O


It is your turn

Enter the x-coordinate [0-2]: 1
Enter the y-coordinate [0-2]: 0
_ _ _ 
X _ _ 
_ _ _ 


The is computer is playing its turn
O _ _ 
X _ _ 
_ _ _ 


It is your turn

Enter the x-coordinate [0-2]: 1
Enter the y-coordinate [0-2]: 2
O _ _ 
X _ X 
_ _ _ 


The is computer is playing its turn
O _ _ 
X O X 
_ _ _ 


It is your turn

Enter the x-coordinate [0-2]: 1
Enter the y-coordinate [0-2]: 0
That coordinate is already taken. Please try again.


It is your turn

Enter the x-coordinate [0-2]: 1
Enter the y-coordinate [0-2]: 2
That coordinate is already taken. Please try again.


It is your turn

Enter the x-coordinate [0-2]: 02
Enter the y-coordinate [0-2]: 1
O _ _ 
X O X 
_ X _ 


The is computer is playing its turn
O _ _ 
X O X 
_ X O 
You have lost!
